<a href="https://colab.research.google.com/github/Nargesmohammadi/API-Endpoints-and-API-Parameters/blob/master/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [2]:
!pip install evaluate -q

In [15]:
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset
import evaluate
metric = evaluate.load("sacrebleu")  # or "rouge", "bleu", etc.
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root

True

In [3]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

print(f"features in cnn_dailymail : {dataset['train'].column_names}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

features in cnn_dailymail : ['article', 'highlights', 'id']


In [17]:
sample = dataset["train"][1]
print(f"""Article except of 500 characters, total length: {len(sample["article"])}): """)
print(sample["article"][:500])
print(f'\nsummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])

Article except of 500 characters, total length: 4051): 
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


## Text Summarization Pipelines

In [23]:

sample_text = dataset["train"][1]["article"][:1000]
# collecting the generated summarizes of each model in dictionary
summaries = {}

## Summarization Baseline

In [24]:
def baseline_summary_three_sent(text):
  return "\n".join(sent_tokenize(text)[:3])

In [25]:
summaries['baseline'] = baseline_summary_three_sent(sample_text)
print(summaries['baseline'])

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."


In [27]:
from transformers import pipeline, set_seed
set_seed(42)
pipe = pipeline('text-generation', model = 'gpt2-medium')
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length = 512, clean_up_tokenization_spaces = True)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [28]:
pipe_out

[{'generated_text': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and

In [29]:
pipe_out[0]['generated_text'][len(gpt2_query) :]

'- The 10th-floor jail, called "the secret prison," was developed specifically for criminally insane people who cannot make bond.\n- It\'s one of only four jails in Miami that houses mentally ill people. The jail was built to house mentally ill people who had "unpredictable behavior"\nTL;DR; The tenth-floor jail was designed to be the secret prison in the world\'s most dangerous jail and has become one where almost all of the mentally ill are housed.\nA mentally ill man, David Garcia-Lopez, has been locked up there for nearly a decade, but he still has his cell phone, despite his wishes. The 10th floor jail has been open for years, but a new administration began installing new surveillance equipment in January 2010. The surveillance cameras are installed above and below ground, a security protocol designed to keep jailers from watching each cell and suspect entering and exiting from another cell. The facility is one of the more violent in Miami. During his five-month incarceration, Gar

In [30]:
summaries['gpt2'] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

## T5

In [31]:
pipe = pipeline('summarization', model = 't5-small')
pipe_out = pipe(sample_text)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0


In [32]:
pipe_out

[{'summary_text': "inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court . most often, they face drug charges or charges of assaulting an officer . mentally ill people become more paranoid, delusional, and less likely to follow dir ."}]

In [34]:
summaries['t5'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))

## Bart

In [35]:
pipe = pipeline('summarization', model = 'facebook/bart-large-cnn')
pipe_out = pipe(sample_text)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [36]:
pipe_out

[{'summary_text': 'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated. Most often, they face drug charges or charges of assaulting an officer. Judge Steven Leifman says the arrests often result from confrontations with police.'}]

In [37]:
summaries['bart'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))

## Pegasus

In [53]:
from huggingface_hub import login
login("")


In [55]:
pipe_pegasus = pipeline('summarization', model = "google/pegasus-cnn_dailymail")
pipe_out = pipe_pegasus(sample_text)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cuda:0


In [56]:
pipe_out

[{'summary_text': 'Mentally ill inmates are housed on the "forgotten floor" of a Miami jail .<n>Judge Steven Leifman says the charges are usually "avoidable felonies"<n>He says the arrests often result from confrontations with police .<n>Mentally ill people often won\'t do what they\'re told when police arrive on the scene .'}]

In [57]:
summaries['pegasus'] = pipe_out[0]['summary_text'].replace(" .<n>", ".\n")

In [ ]:
## Comparing Different Summaries

In [58]:
print("GROUND TRUTH")
print(dataset['train'][1]['highlights'])

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."
GPT2
- The 10th-floor jail, called "the secret prison," was developed specifically for criminally insane people who cannot make bond.
- It's one of only four jails in Miami that houses mentally ill people.
The jail was built 

## SacreBLEU

In [64]:
import evaluate
bleu_metric = evaluate.load("sacrebleu")


In [66]:
import numpy as np

# Add one example at a time (prediction is a string, reference is a list of strings)
bleu_metric.add(
    prediction=summaries['pegasus'],
    reference=[dataset['train'][1]['highlights']]
)

# Compute BLEU with smoothing options
results = bleu_metric.compute(smooth_method='floor', smooth_value=0)

# Round precisions for better display
results['precision'] = [np.round(p, 2) for p in results['precisions']]

# Display results as a DataFrame
df = pd.DataFrame.from_dict(results, orient='index', columns=['Value'])
print(df)

                                                        Value
score                                               24.873444
counts                                        [29, 16, 12, 7]
totals                                       [58, 57, 56, 55]
precisions  [50.0, 28.07017543859649, 21.428571428571427, ...
bp                                                        1.0
sys_len                                                    58
ref_len                                                    57
precision                         [50.0, 28.07, 21.43, 12.73]


##ROUGE

In [68]:
rouge_metric = evaluate.load('rouge')

In [71]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
reference = dataset['train'][1]['highlights']
records = []
for model_name in summaries:
    rouge_metric.add(
        prediction=summaries[model_name],
        references=[reference])
scores = rouge_metric.compute()
for model_name in summaries:
    rouge_metric = evaluate.load('rouge')  # reload metric fresh each time
    rouge_metric.add(
        prediction=summaries[model_name],
        references=[reference]
    )
    score = rouge_metric.compute()
    rouge_dict = {rn: score[rn] for rn in rouge_names}
    print(f"{model_name} rouge scores:", rouge_dict)
    records.append(rouge_dict)
df = pd.DataFrame.from_records(records, index=summaries.keys())
print(df)

baseline rouge scores: {'rouge1': np.float64(0.365079365079365), 'rouge2': np.float64(0.14516129032258066), 'rougeL': np.float64(0.20634920634920634), 'rougeLsum': np.float64(0.2857142857142857)}
gpt2 rouge scores: {'rouge1': np.float64(0.16556291390728475), 'rouge2': np.float64(0.02666666666666667), 'rougeL': np.float64(0.09933774834437085), 'rougeLsum': np.float64(0.16556291390728475)}
t5 rouge scores: {'rouge1': np.float64(0.19780219780219782), 'rouge2': np.float64(0.022471910112359546), 'rougeL': np.float64(0.13186813186813187), 'rougeLsum': np.float64(0.19780219780219782)}
bart rouge scores: {'rouge1': np.float64(0.3655913978494624), 'rouge2': np.float64(0.13186813186813184), 'rougeL': np.float64(0.2150537634408602), 'rougeLsum': np.float64(0.3225806451612903)}
pegasus rouge scores: {'rouge1': np.float64(0.5), 'rouge2': np.float64(0.24489795918367346), 'rougeL': np.float64(0.36000000000000004), 'rougeLsum': np.float64(0.46)}
            rouge1    rouge2    rougeL  rougeLsum
baseli

## Evaluating on the Test set of the CNN/DailyMail Dataset

In [74]:
def calculate_metric_on_baseline_test_ds(dataset, metric, column_text='article', column_summary='highlights'):
    summaries = [baseline_summary_three_sent(text) for text in dataset[column_text]]
    references = [str(ref) for ref in dataset[column_summary]]  # ensure list of strings
    metric.add_batch(predictions=summaries, references=references)
    score = metric.compute()
    return score

In [75]:
test_sampled = dataset['train'].shuffle(seed=42).select(range(1000))
score = calculate_metric_on_baseline_test_ds(test_sampled, rouge_metric)

# No .mid.fmeasure now; just access floats directly
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame.from_dict(rouge_dict, orient='index', columns=['baseline']).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.253995,0.100642,0.165754,0.231571


## Strategy to calculate the ROUGE Metric on test dataset for the othe Models

In [79]:
from tqdm import tqdm
import torch
device = "cuda" if torch.cuda.is_available() else 'cpu'
def generate_batch_sized_chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i: 1 +batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=1, device = device, column_text = 'article', column_summary = 'highlights'):
    article_bathces = list(generate_batch_sized_chunks(dataset[column_text]))
    target_bathces = list(generate_batch_sized_chunks(dataset[column_summary]))

    for article_batch, target_batch in tqdm(zip(article_bathces, target_bathces), total = len(article_bathces)):
        inputs = tokenizer(article_bathces_batch, max_len=1024, truncation = True, padding = 'max_length', return_lensor = 'pt')
        summaries = model.generate(input_ids = inputs['input_ids'].to(device), attention_mask = inputs['attention_mask'].to(device),
                                  length_penalty = 0.8, num_beams = 8, max_length = 128
                                  )
        decode_summaries = [tokenizer.decode(s, skip_special_tokens = True, clean_up_tokenization_spaces = True) for s in summaries]
        decode_summaries = [d.replace('<n>', "") for d in decode_summaries]

        metric.add_batch(predictions = decoded_summaries, references = target_batch)

    score = metric.compute()
    return score

In [99]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_ckpt = 'google/pegasus-cnn-dailymail'
token = ''

# Pass token to both tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_auth_token=token)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt, use_auth_token=token)
model_pegasus.to(device)

score = calculate_metric_on_test_ds(test_sampled, rouge_metric, model_pegasus, tokenizer, batch_size)
df = pd.DataFrame(score, index=['pegasus'])
print(df)

OSError: There was a specific connection error when trying to load google/pegasus-cnn-dailymail:
401 Client Error: Unauthorized for url: https://huggingface.co/google/pegasus-cnn-dailymail/resolve/main/config.json (Request ID: Root=1-682bab5d-7598f7810adb6d71048aa6dc;60d41de6-fb7c-49b7-9f04-e8f96df7db2a)

Invalid credentials in Authorization header